This is the 3.4 notebook

Cell 0 — Context

In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA silver")

Cell 1 — Imports

In [0]:
from pyspark.sql.functions import col

Cell 2 — Load curated table

In [0]:
base_df = spark.table("workspace.silver.stedi_step_curated")

Cell 3 — Final projection

In [0]:
from pyspark.sql.functions import col

final_df = base_df.select(
    col("timestamp"),
    col("device_id"),
    col("distance_cm"),
    col("step_label"),
    col("source").alias("source_label")
)


One sanity check

In [0]:
final_df.explain(True)

Cell 10 — Final cell

In [0]:
(
    final_df
    .write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("workspace.silver.labeled_step_test")
)


In [0]:
spark.table("workspace.silver.labeled_step_test").count()

SQL Query Test

In [0]:
%sql
-- Steps vs. No-Steps
SELECT step_label, COUNT(*)
FROM labeled_step_test
GROUP BY step_label;

-- Invalid or missing labels
SELECT *
FROM labeled_step_test
WHERE step_label NOT IN ('step', 'no_step')
  OR step_label IS NULL


LIMIT 50;

-- Source label counts
SELECT source_label, COUNT(*)
FROM labeled_step_test
GROUP BY source_label;

-- Invalid source labels
SELECT *
FROM labeled_step_test
WHERE source_label NOT IN ('device', 'step')
   OR source_label IS NULL
LIMIT 50;